In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
class DeepWeedsDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = f"{self.img_dir}/{self.data.iloc[idx, 0]}"
        image = Image.open(img_path).convert("RGB")
        label = int(self.data.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])


In [7]:
train_dataset = DeepWeedsDataset(
    csv_file="labels/labels.csv",
    img_dir="images",
    transform=train_transform
)

test_dataset = DeepWeedsDataset(
    csv_file="labels/test_subset1.csv",
    img_dir="images",
    transform=test_transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(train_dataset.data.iloc[:,1].unique())


In [8]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)


def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


In [9]:
def get_resnet18(num_classes, partial=True):
    model = models.resnet18(pretrained=True)

    if partial:
        for param in model.parameters():
            param.requires_grad = False

    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)


In [10]:
results = {}

for mode in ["partial", "full"]:
    print(f"\nTraining with {mode} fine-tuning")

    model = get_resnet18(num_classes, partial=(mode=="partial"))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

    for epoch in range(5):
        loss = train_one_epoch(model, train_loader, optimizer, criterion)
        print(f"Epoch {epoch+1}: Loss = {loss:.4f}")

    y_true, y_pred = evaluate(model, test_loader)

    results[mode] = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average="weighted"),
        "recall": recall_score(y_true, y_pred, average="weighted"),
        "f1": f1_score(y_true, y_pred, average="weighted"),
        "cm": confusion_matrix(y_true, y_pred)
    }



Training with partial fine-tuning
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100.0%


Epoch 1: Loss = 1.0744
Epoch 2: Loss = 0.8580
Epoch 3: Loss = 0.8070
Epoch 4: Loss = 0.7857
Epoch 5: Loss = 0.7763

Training with full fine-tuning


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Loss = 0.8654
Epoch 2: Loss = 0.5837
Epoch 3: Loss = 0.4660
Epoch 4: Loss = 0.4047
Epoch 5: Loss = 0.3639


In [11]:
for mode, metrics in results.items():
    print(f"\n{mode.upper()} FINE-TUNING")
    for k,v in metrics.items():
        if k != "cm":
            print(f"{k}: {v:.4f}")



PARTIAL FINE-TUNING
accuracy: 0.7585
precision: 0.7617
recall: 0.7585
f1: 0.7499

FULL FINE-TUNING
accuracy: 0.9047
precision: 0.9079
recall: 0.9047
f1: 0.9019


In [12]:
for mode, metrics in results.items():
    print(f"\n{mode.upper()} FINE-TUNING")
    for k,v in metrics.items():
        if k != "cm":
            print(f"{k}: {v:.4f}")



PARTIAL FINE-TUNING
accuracy: 0.7585
precision: 0.7617
recall: 0.7585
f1: 0.7499

FULL FINE-TUNING
accuracy: 0.9047
precision: 0.9079
recall: 0.9047
f1: 0.9019


### Model Choice
ResNet-18 was chosen for the DeepWeeds dataset because weed classification involves complex natural images with variations in background, lighting, and plant structure. The residual architecture allows the model to learn strong spatial and texture-based features while maintaining stable training. ResNet-18 provides a good balance between performance and computational efficiency, making it suitable for handling outdoor images without requiring very deep networks. Transfer learning from ImageNet further improves generalization on plant images.